# Building Mapping Functions

In [1]:
import pandas as pd
import folium
import os
import json
import geopy.distance
import geopandas
from branca.colormap import linear
import pickle
import requests

# Build Map of Any Hour, 2015-2019

In [2]:
# Bring in all trip data
full_df = pd.read_csv('data/trips/full.csv')

In [3]:
# Convert the start time to datetime
full_df['start_date'] = pd.to_datetime(full_df['start_date'])

In [4]:
# Use Grouper to group by neighborhood and hour
grouped_full_df = full_df.groupby([pd.Grouper(key='start_date', freq='H'), "start_neighborhood"])["day_of_week"].count().unstack(fill_value=0).stack()

In [5]:
# Select only rides after 1/1/2015 to meet GitHub file size limits
grouped_2015_df = grouped_full_df['1/1/2015 12am':]

In [6]:
# Save data for use with Flask
with open('data/grouped_2015_df.pickle', 'wb') as handle:
    pickle.dump(grouped_2015_df, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [7]:
# Use GeoPandas to create new neighborhood object
neighborhoods = geopandas.read_file('data/geography/Neighborhood_Clusters.geojson', driver='GeoJSON')

In [8]:
# Save data for use with Flask
with open('data/neighborhoods.pickle', 'wb') as handle:
    pickle.dump(neighborhoods, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [11]:
# Merge selected counts into neighborhood DataFrame
neighborhoods = neighborhoods.merge(grouped_2015_df['2015-01-01 00:00:00'].rename('counts'), how='outer', left_on='NBH_NAMES', right_index=True)

In [12]:
# Replace null values with zeroes
neighborhoods = neighborhoods.fillna({'counts':0})

In [13]:
def build_map(date, hour):
    # Load ride data and geographic data
    with open('data/grouped_2015_df.pickle', 'rb') as handle:
        grouped_2015_df = pickle.load(handle)
    with open('data/neighborhoods.pickle', 'rb') as handle:
        neighborhoods = pickle.load(handle)
    # Use input variables to locate requested date and hour
    neighborhoods = neighborhoods.merge(grouped_2015_df['{} {}'.format(date, hour)].rename('counts'), how='outer', left_on='NBH_NAMES', right_index=True)
    # Fill empty rows with zero, as no rides took place if no data is present
    neighborhoods = neighborhoods.fillna({'counts':0})
    # Set the color map, using the minimum as the lightest color and maximum as the darkest color
    colormap = linear.YlGn_09.scale(neighborhoods['counts'].min(),neighborhoods['counts'].max())
    # Set the map's location, zoom, and tile set
    m = folium.Map(location=[38.9072, -77.0369],
    zoom_start=11)
    # Set the style for each row, hardcoding "white" for values of zero.
    style_function=lambda feature: {
        'fillColor': 'white' if feature['properties']['counts'] == 0 else colormap(feature['properties']['counts']),
        'color': 'black',
        'weight': 2,
        'fillOpacity': 0.9
    }
    # Tell folium what data and style to use, what to include in the tooltip, and what aliases to use for the tooltip
    neighborhood_geo = folium.GeoJson(
    neighborhoods,
    name='DC Neighborhoods',
    style_function=style_function,
    tooltip=folium.GeoJsonTooltip(
        fields=['NBH_NAMES', 'counts'],
        aliases=['Cluster', 'Rides Started'],
        localize=True)
    ).add_to(m)
    # Include the colormap as a legend
    colormap.add_to(m)
    # Add layer control functionality to the map
    folium.LayerControl().add_to(m)
    return m

In [14]:
build_map('1/20/2018', '5pm')

# Identify Current Docks for Dynamic Pricing

## Bring In Station Data

In [15]:
# Bring in current station information data
url = 'https://gbfs.capitalbikeshare.com/gbfs/fr/station_information.json'
resp = requests.get(url=url)
station_data = resp.json()

In [16]:
# Parse the data to build the relevant dataframe
station_df = pd.DataFrame.from_records(station_data['data']['stations'])

In [17]:
# Select the relevant columns
station_df = station_df[['capacity', 'lat', 'lon', 'name', 'region_id', 'short_name', 'station_id']]

## Bring In Current Station Status Data

In [18]:
# Bring in current station status data
url = 'https://gbfs.capitalbikeshare.com/gbfs/fr/station_status.json'
resp = requests.get(url=url)
status_data = resp.json()

In [19]:
# Parse the data to build the relevant dataframe
status_df = pd.DataFrame.from_records(status_data['data']['stations'])

In [20]:
# Add current bike data to station dataframe
station_df['available_bikes'] = status_df['num_bikes_available']

In [21]:
# Add colors that depend on number of bikes currently available 
station_df['alert_marker_color'] = pd.cut(station_df['available_bikes'], bins=[-1, 3, 7, 100], 
                              labels=['red', 'yellow', 'green'])

In [22]:
# Select only stations in Washington, DC
station_df = station_df.loc[station_df['region_id'] == 42]

In [23]:
# Set map location and zoom
m = folium.Map(location=[38.9072, -77.0369],
    zoom_start=11)

# Add markers to map using lattitude, longitude, and color
# Add tooltip including station name and number of bikes available
for index, row in station_df.iterrows():
#     if row['alert_marker_color'] != 'blue':
        folium.CircleMarker([row['lat'], row['lon']],
                    radius=4, color=row['alert_marker_color'], tooltip=['Station: {}'.format(row['name']), 'Available bikes: {}'.format(row['available_bikes'])]).add_to(m)
    
m

# Build Dynamic Pricing Map

In [24]:
# Create dataframe that includes only bikes with "red" label
red_df = station_df.loc[station_df['alert_marker_color'] == 'red']

In [25]:
m = folium.Map(location=[38.9072, -77.0369],
    zoom_start=13)

# start = folium.FeatureGroup(name='Current Status', hidden=True)
dynamic = folium.FeatureGroup(name='Proposed Dynamic Pricing')


# for index, row in station_df.iterrows():
#         folium.CircleMarker(location=[row['lat'], row['lon']],
#                     radius=4, color=row['alert_marker_color'], tooltip=['Station: {}'.format(row['name']), 'Available bikes: {}'.format(row['available_bikes'])]).add_to(start)

# Find docks with 6 or more bikes under .25 miles from location, draw line that connects them, and add circular region around low dock
for index, row in red_df.iterrows():
    start_lat = (row['lat'])
    start_lon = (row['lon'])
    start_point = (row['lat'], row['lon'])
    start_name = row['name']
    start_status = row['alert_marker_color']
    start_available = row['available_bikes']
    for index, row in station_df.iterrows():
        end_lat = row['lat']
        end_lon = row['lon']
        end_name = row['name']
        end_point = (row['lat'], row['lon'])
        end_status = row['alert_marker_color']
        dist = geopy.distance.geodesic(start_point, end_point).miles
        if 0 < dist < .25:
            if end_status == 'green':
                folium.Circle(location = [start_lat, start_lon], radius=(dist*1609.34/2), fill_color='#3186cc', fill=True, popup='Proposed Discount Region').add_to(dynamic)
                folium.CircleMarker(end_point,
                    radius=5,
                    color=end_status,
                    tooltip=['Station: {}'.format(end_name), 'Available bikes: {}'.format(row['available_bikes'])]
                    ).add_to(dynamic)
                folium.CircleMarker(start_point,
                    radius=5,
                    color=start_status,
                    tooltip=['Station: {}'.format(start_name), 'Available bikes: {}'.format(start_available)]
                    ).add_to(dynamic)
                folium.PolyLine([start_point, end_point], color='black').add_to(dynamic)
                break

dynamic.add_to(m)
# start.add_to(m)
lc = folium.LayerControl().add_to(m)

m

In [28]:
def build_dynamic_pricing_map():   
    m = folium.Map(location=[38.9072, -77.0369],
    zoom_start=13, tiles='CartoDB dark_matter')

    start = folium.FeatureGroup(name='Current Status', show=False)
    dynamic = folium.FeatureGroup(name='Proposed Dynamic Pricing')


    for index, row in station_df.iterrows():
            folium.CircleMarker(location=[row['lat'], row['lon']],
                        radius=4, color=row['alert_marker_color'], tooltip=['Station: {}'.format(row['name']), 'Available bikes: {}'.format(row['available_bikes'])]).add_to(start)

# Find docks with 6 or more bikes under .25 miles from location
    for index, row in red_df.iterrows():
        start_lat = (row['lat'])
        start_lon = (row['lon'])
        start_point = (row['lat'], row['lon'])
        start_name = row['name']
        start_status = row['alert_marker_color']
        start_available = row['available_bikes']
        for index, row in station_df.iterrows():
            end_lat = row['lat']
            end_lon = row['lon']
            end_name = row['name']
            end_point = (row['lat'], row['lon'])
            end_status = row['alert_marker_color']
            dist = geopy.distance.geodesic(start_point, end_point).miles
            if 0 < dist < .25:
                if end_status == 'green':
                    folium.Circle(location = [start_lat, start_lon], radius=(dist*1609.34/2), fill_color='#3186cc', fill=True, popup='Proposed Discount Region').add_to(dynamic)
                    folium.CircleMarker(end_point,
                        radius=5,
                        color=end_status,
                        tooltip=['Station: {}'.format(end_name), 'Available bikes: {}'.format(row['available_bikes'])]
                        ).add_to(dynamic)
                    folium.CircleMarker(start_point,
                        radius=5,
                        color=start_status,
                        tooltip=['Station: {}'.format(start_name), 'Available bikes: {}'.format(start_available)]
                        ).add_to(dynamic)
                    folium.PolyLine([start_point, end_point], color='white').add_to(dynamic)
                    break
    dynamic.add_to(m)
    start.add_to(m)
    lc = folium.LayerControl().add_to(m)
    return m

In [29]:
build_dynamic_pricing_map()